# Australian Property Analytics Tool

## Free Property Market Analysis System

This notebook provides a complete property analytics solution for the Australian market using **free data sources only**:

1. **📊 Data Generation**: Realistic property data based on Australian market patterns
2. **🏛️ ABS Integration**: Australian Bureau of Statistics socio-economic data
3. **🧹 Data Processing**: Cleaning, normalization, and geospatial analysis
4. **📈 KPI Computation**: Market indicators and regional analysis
5. **🤖 ML Modeling**: XGBoost/Random Forest valuation models
6. **🔮 Monte Carlo Simulation**: Future price forecasting
7. **📊 Visualization**: Interactive charts, maps, and dashboards

**✅ No API keys required - completely free to use!**

---

## 1. Setup and Configuration

Import required libraries and configure the analysis environment.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import warnings
import os
import sys
from datetime import datetime, timedelta

# Add scripts directory to path
sys.path.append('../scripts')

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Core libraries imported successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🆓 Using free data sources only - no API keys required!")

In [ ]:
# Import custom modules (ABS and statistics-based only)
try:
    from data_fetcher import create_realistic_property_data, fetch_abs_socioeconomic_data
    from data_processor import PropertyDataProcessor, calculate_property_kpis
    from ml_models import PropertyValuationModel, identify_overvalued_properties
    from monte_carlo import MonteCarloPropertySimulation, run_portfolio_simulation
    from visualization import PropertyVisualizationSuite
    print("✅ Custom modules imported successfully")
    print("🏛️ Using ABS data and realistic property generation functions")
except ImportError as e:
    print(f"⚠️ Import warning: {e}")
    print("Some features may not be available. Please install required packages with:")
    print("pip install -r requirements.txt")

In [ ]:
# Configuration - No API keys needed!
CONFIG = {
    'ANALYSIS_SUBURBS': ['Sydney', 'Melbourne', 'Brisbane', 'Perth', 'Adelaide'],
    'PROPERTY_TYPES': ['House', 'Unit', 'Townhouse'],
    'SIMULATION_YEARS': 5,
    'SIMULATION_RUNS': 1000,
    'MODEL_TYPE': 'xgboost',  # or 'random_forest'
    'RANDOM_STATE': 42,
    'NUM_PROPERTIES': 500  # Number of sample properties to generate
}

# Create data directories
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../data/outputs', exist_ok=True)

print("✅ Configuration loaded")
print(f"🏙️ Analysis suburbs: {', '.join(CONFIG['ANALYSIS_SUBURBS'])}")
print(f"🏠 Property types: {', '.join(CONFIG['PROPERTY_TYPES'])}")
print(f"🔢 Sample size: {CONFIG['NUM_PROPERTIES']} properties")

## 2. Data Generation

Generate realistic property data based on Australian market patterns and load ABS socio-economic statistics.

In [ ]:
# Generate realistic property data based on Australian market patterns
print("📊 Generating realistic property data based on Australian market patterns...")
print("🆓 No API calls required - using statistical models!")

# Use the enhanced data generation function
property_data = create_realistic_property_data(
    suburbs=CONFIG['ANALYSIS_SUBURBS'],
    property_types=CONFIG['PROPERTY_TYPES'],
    num_properties=CONFIG['NUM_PROPERTIES'],
    random_state=CONFIG['RANDOM_STATE']
)

print(f"✅ Generated {len(property_data)} property records")
print(f"📈 Price range: ${property_data['price'].min():,.0f} - ${property_data['price'].max():,.0f}")
print(f"🏠 Property types: {property_data['property_type'].value_counts().to_dict()}")
print(f"🏙️ Suburbs: {property_data['suburb'].value_counts().to_dict()}")

# Display sample
print("\n📋 Sample property data:")
display(property_data.head())

In [ ]:
# Fetch socio-economic data from ABS
print("🏛️ Loading socio-economic data based on ABS statistics...")
print("📊 Using Australian Bureau of Statistics patterns - no API required!")

# Load enhanced socio-economic data
socioeconomic_data = fetch_abs_socioeconomic_data(CONFIG['ANALYSIS_SUBURBS'])

print(f"✅ Loaded socio-economic data for {len(socioeconomic_data)} regions")
print(f"📋 Available metrics: {list(socioeconomic_data.columns)}")

print("\n📈 ABS-based socio-economic indicators:")
display(socioeconomic_data)

## 3. Data Processing and Cleaning

Clean, normalize, and merge the datasets for analysis.

In [ ]:
# Initialize data processor
processor = PropertyDataProcessor()

# Clean property data
print("🧹 Cleaning property data...")
clean_property_data = processor.clean_property_data(property_data)

# Merge with socio-economic data
print("🔗 Merging with ABS socio-economic data...")
merged_data = processor.merge_with_socioeconomic_data(clean_property_data, socioeconomic_data)

# Create geospatial features
print("🗺️ Creating geospatial features...")
geo_data = processor.create_geospatial_features(merged_data)
geo_data = processor.calculate_distance_features(geo_data)

print(f"✅ Data processing complete")
print(f"📊 Final dataset shape: {geo_data.shape}")
print(f"📋 Columns: {list(geo_data.columns)}")

# Display processed data sample
print("\n📋 Processed data sample:")
display(geo_data[['suburb', 'property_type', 'price', 'bedrooms', 'MEDIAN_INCOME', 'distance_to_cbd_km']].head())

## 4. KPI Computation

Calculate key property market indicators and regional statistics.

In [ ]:
# Calculate property KPIs
print("📈 Calculating property market KPIs...")

kpis = calculate_property_kpis(geo_data)

# Display overall market metrics
print("\n🏠 OVERALL MARKET METRICS")
print(f"Median Price: ${kpis['median_price']:,.0f}")
print(f"Mean Price: ${kpis['mean_price']:,.0f}")
print(f"Price Std Dev: ${kpis['price_std']:,.0f}")

# Suburb-level analysis
print("\n🏙️ SUBURB-LEVEL ANALYSIS")
suburb_stats = geo_data.groupby('suburb').agg({
    'price': ['count', 'median', 'mean'],
    'MEDIAN_INCOME': 'first',
    'UNEMPLOYMENT_RATE': 'first'
}).round(0)

suburb_stats.columns = ['Count', 'Median_Price', 'Mean_Price', 'Median_Income', 'Unemployment_Rate']
suburb_stats['Affordability_Ratio'] = suburb_stats['Median_Price'] / suburb_stats['Median_Income']

display(suburb_stats)

# Property type analysis
print("\n🏠 PROPERTY TYPE ANALYSIS")
type_stats = geo_data.groupby('property_type')['price'].agg(['count', 'median', 'mean']).round(0)
display(type_stats)

## 5. Machine Learning Valuation Model

Build and train XGBoost/Random Forest models for property valuation.

In [ ]:
# Initialize and train valuation model
print(f"🤖 Training {CONFIG['MODEL_TYPE']} valuation model...")

model = PropertyValuationModel(model_type=CONFIG['MODEL_TYPE'])

# Train the model
training_metrics = model.train(
    geo_data, 
    target_col='price',
    random_state=CONFIG['RANDOM_STATE']
)

print("\n📊 MODEL PERFORMANCE METRICS")
print(f"Train R²: {training_metrics['train_r2']:.3f}")
print(f"Test R²: {training_metrics['test_r2']:.3f}")
print(f"Train RMSE: ${training_metrics['train_rmse']:,.0f}")
print(f"Test RMSE: ${training_metrics['test_rmse']:,.0f}")
print(f"Test MAE: ${training_metrics['test_mae']:,.0f}")
print(f"Features used: {training_metrics['feature_count']}")

# Feature importance
print("\n🔍 TOP 10 MOST IMPORTANT FEATURES")
feature_importance = model.get_feature_importance()
if not feature_importance.empty:
    display(feature_importance.head(10))
else:
    print("Feature importance not available for this model type")

In [ ]:
# Identify over/undervalued properties
print("🔍 Identifying over/undervalued properties...")

valuation_analysis = identify_overvalued_properties(
    geo_data, 
    model, 
    threshold=0.15  # 15% threshold
)

# Valuation summary
valuation_summary = valuation_analysis['valuation_status'].value_counts()
print("\n💰 VALUATION ANALYSIS SUMMARY")
for status, count in valuation_summary.items():
    percentage = (count / len(valuation_analysis)) * 100
    print(f"{status}: {count} properties ({percentage:.1f}%)")

# Show some examples
print("\n📋 SAMPLE OVERVALUED PROPERTIES")
overvalued = valuation_analysis[valuation_analysis['valuation_status'] == 'Overvalued']
if len(overvalued) > 0:
    sample_overvalued = overvalued[['suburb', 'property_type', 'price', 'predicted_price', 'price_difference_pct']].head()
    display(sample_overvalued)

print("\n📋 SAMPLE UNDERVALUED PROPERTIES")
undervalued = valuation_analysis[valuation_analysis['valuation_status'] == 'Undervalued']
if len(undervalued) > 0:
    sample_undervalued = undervalued[['suburb', 'property_type', 'price', 'predicted_price', 'price_difference_pct']].head()
    display(sample_undervalued)
else:
    print("No undervalued properties found with current threshold")

## 6. Monte Carlo Price Simulation

Forecast future property prices using Monte Carlo simulation with economic variables.

In [ ]:
# Monte Carlo simulation for a sample property
sample_price = geo_data['price'].median()

print(f"🎯 Running Monte Carlo simulation for property valued at ${sample_price:,.0f}")
print(f"⏱️ Simulation parameters: {CONFIG['SIMULATION_YEARS']} years, {CONFIG['SIMULATION_RUNS']} runs")

# Initialize simulator
simulator = MonteCarloPropertySimulation(
    base_price=sample_price,
    simulation_years=CONFIG['SIMULATION_YEARS'],
    num_simulations=CONFIG['SIMULATION_RUNS']
)

# Run simulation
simulation_results = simulator.run_simulation()
simulation_stats = simulator.get_simulation_statistics()

print("\n📊 SIMULATION RESULTS")
print(f"Expected price after {CONFIG['SIMULATION_YEARS']} years: ${simulation_stats['final_price_mean']:,.0f}")
print(f"Median forecast: ${simulation_stats['final_price_median']:,.0f}")
print(f"Expected annual return: {simulation_stats['annual_return_pct']:.2f}%")
print(f"Probability of gain: {simulation_stats['probability_gain']:.1%}")
print(f"Probability of loss: {simulation_stats['probability_loss']:.1%}")

print("\n📈 CONFIDENCE INTERVALS")
print(f"5th percentile: ${simulation_stats['percentile_5']:,.0f}")
print(f"25th percentile: ${simulation_stats['percentile_25']:,.0f}")
print(f"75th percentile: ${simulation_stats['percentile_75']:,.0f}")
print(f"95th percentile: ${simulation_stats['percentile_95']:,.0f}")

In [ ]:
# Portfolio simulation example
print("🏠 Running portfolio simulation...")

# Select sample portfolio (median prices by suburb)
portfolio_prices = geo_data.groupby('suburb')['price'].median().values

portfolio_stats = run_portfolio_simulation(
    portfolio_prices.tolist(),  # Convert numpy array to list
    simulation_years=CONFIG['SIMULATION_YEARS'],
    num_simulations=500  # Reduced for faster computation
)

print("\n💼 PORTFOLIO SIMULATION RESULTS")
print(f"Initial portfolio value: ${portfolio_stats['initial_portfolio_value']:,.0f}")
print(f"Expected portfolio value after {CONFIG['SIMULATION_YEARS']} years: ${portfolio_stats['final_portfolio_mean']:,.0f}")
print(f"Portfolio annual return: {portfolio_stats['annual_return_pct']:.2f}%")
print(f"Value at Risk (5%): ${portfolio_stats['value_at_risk_5pct']:,.0f}")

print("\n🏘️ INDIVIDUAL PROPERTY RETURNS")
for i, (prop_id, stats) in enumerate(portfolio_stats['individual_properties'].items()):
    if i < len(CONFIG['ANALYSIS_SUBURBS']):
        suburb = CONFIG['ANALYSIS_SUBURBS'][i]
        print(f"{suburb}: {stats['annual_return_pct']:.2f}% annual return")

## 7. Interactive Visualizations

Create interactive charts, maps, and dashboards for data exploration.

In [ ]:
# Create visualizations
print("📊 Creating visualizations...")

try:
    import plotly.express as px
    import plotly.graph_objects as go
    
    # Initialize visualization suite
    viz = PropertyVisualizationSuite()
    
    # Price distribution by property type
    price_dist_fig = viz.create_price_distribution_chart(
        geo_data, 
        group_by='property_type',
        price_col='price'
    )
    price_dist_fig.show()
    
    print("✅ Interactive plotly charts created successfully")
    
except ImportError:
    print("⚠️ Plotly not available. Creating matplotlib visualizations...")
    
    # Create simple matplotlib visualization as fallback
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Price by suburb
    geo_data.boxplot(column='price', by='suburb', ax=axes[0,0])
    axes[0,0].set_title('Price Distribution by Suburb')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Price by property type
    geo_data.boxplot(column='price', by='property_type', ax=axes[0,1])
    axes[0,1].set_title('Price Distribution by Property Type')
    
    # Price vs bedrooms
    geo_data.groupby('bedrooms')['price'].median().plot(kind='bar', ax=axes[1,0])
    axes[1,0].set_title('Median Price by Bedrooms')
    
    # Price vs distance to CBD
    axes[1,1].scatter(geo_data['distance_to_cbd_km'], geo_data['price'], alpha=0.6)
    axes[1,1].set_xlabel('Distance to CBD (km)')
    axes[1,1].set_ylabel('Price ($)')
    axes[1,1].set_title('Price vs Distance to CBD')
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Matplotlib charts created successfully")

In [ ]:
# Valuation gauge for a sample property
try:
    sample_property = valuation_analysis.iloc[0]
    
    gauge_fig = viz.create_valuation_gauge(
        actual_price=sample_property['price'],
        predicted_price=sample_property['predicted_price'],
        property_address=f"{sample_property['suburb']} - {sample_property['property_type']}"
    )
    gauge_fig.show()
    
except (NameError, ImportError):
    print("📊 Creating text-based valuation summary...")
    
    # Text-based valuation summary
    sample_property = valuation_analysis.iloc[0]
    valuation_pct = sample_property['price_difference_pct']
    
    print(f"\n🏠 SAMPLE PROPERTY VALUATION")
    print(f"Location: {sample_property['suburb']}")
    print(f"Type: {sample_property['property_type']}")
    print(f"Actual Price: ${sample_property['price']:,.0f}")
    print(f"Model Prediction: ${sample_property['predicted_price']:,.0f}")
    print(f"Difference: {valuation_pct:.1f}%")
    print(f"Status: {sample_property['valuation_status']}")

In [ ]:
# Monte Carlo simulation visualization
try:
    # Simulation forecast chart
    forecast_fig = simulator.plot_simulation_results()
    forecast_fig.show()
    
    # Final price distribution
    dist_fig = simulator.plot_final_price_distribution()
    dist_fig.show()
    
    print("✅ Interactive simulation charts created")
    
except (NameError, ImportError):
    print("📊 Creating matplotlib simulation charts...")
    
    # Simple matplotlib version
    import matplotlib.pyplot as plt
    
    final_year_col = f'Year_{CONFIG["SIMULATION_YEARS"]}'
    final_prices = simulation_results[final_year_col]
    
    plt.figure(figsize=(12, 5))
    
    # Histogram of final prices
    plt.subplot(1, 2, 1)
    plt.hist(final_prices, bins=50, alpha=0.7, edgecolor='black')
    plt.axvline(sample_price, color='green', linestyle='--', label='Current Price')
    plt.axvline(final_prices.mean(), color='red', linestyle='--', label='Expected Price')
    plt.xlabel('Property Price ($)')
    plt.ylabel('Frequency')
    plt.title(f'Price Distribution After {CONFIG["SIMULATION_YEARS"]} Years')
    plt.legend()
    
    # Simulation paths (sample)
    plt.subplot(1, 2, 2)
    years = list(range(CONFIG['SIMULATION_YEARS'] + 1))
    for i in range(min(50, len(simulation_results))):
        prices = [simulation_results.iloc[i][f'Year_{year}'] for year in years]
        plt.plot(years, prices, alpha=0.1, color='blue')
    
    # Add mean forecast
    mean_prices = [simulation_results[f'Year_{year}'].mean() for year in years]
    plt.plot(years, mean_prices, color='red', linewidth=3, label='Mean Forecast')
    
    plt.xlabel('Years')
    plt.ylabel('Property Price ($)')
    plt.title('Monte Carlo Price Forecasts')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Matplotlib simulation charts created")

## 8. Summary and Export Results

Generate comprehensive summary report and export results for further analysis.

In [ ]:
# Generate comprehensive summary report
print("📝 PROPERTY ANALYTICS SUMMARY REPORT")
print("=" * 60)

print(f"\n📊 DATASET OVERVIEW")
print(f"Total properties analyzed: {len(geo_data):,}")
print(f"Analysis period: {geo_data['date_listed'].min().strftime('%Y-%m-%d')} to {geo_data['date_listed'].max().strftime('%Y-%m-%d')}")
print(f"Suburbs covered: {', '.join(CONFIG['ANALYSIS_SUBURBS'])}")
print(f"Property types: {', '.join(CONFIG['PROPERTY_TYPES'])}")
print(f"Data source: 🆓 Free - ABS statistics + realistic generation")

print(f"\n💰 MARKET OVERVIEW")
print(f"Overall median price: ${geo_data['price'].median():,.0f}")
print(f"Price range: ${geo_data['price'].min():,.0f} - ${geo_data['price'].max():,.0f}")
print(f"Most expensive suburb: {geo_data.groupby('suburb')['price'].median().idxmax()}")
print(f"Most affordable suburb: {geo_data.groupby('suburb')['price'].median().idxmin()}")

print(f"\n🤖 MODEL PERFORMANCE")
print(f"Model type: {CONFIG['MODEL_TYPE'].title()}")
print(f"Test R² Score: {training_metrics['test_r2']:.3f}")
print(f"Test RMSE: ${training_metrics['test_rmse']:,.0f}")
print(f"Mean Absolute Error: ${training_metrics['test_mae']:,.0f}")

print(f"\n🔍 VALUATION INSIGHTS")
overvalued_count = len(valuation_analysis[valuation_analysis['valuation_status'] == 'Overvalued'])
undervalued_count = len(valuation_analysis[valuation_analysis['valuation_status'] == 'Undervalued'])
fair_value_count = len(valuation_analysis[valuation_analysis['valuation_status'] == 'Fair Value'])

print(f"Overvalued properties: {overvalued_count} ({overvalued_count/len(valuation_analysis)*100:.1f}%)")
print(f"Undervalued properties: {undervalued_count} ({undervalued_count/len(valuation_analysis)*100:.1f}%)")
print(f"Fair value properties: {fair_value_count} ({fair_value_count/len(valuation_analysis)*100:.1f}%)")

print(f"\n🔮 FORECAST SUMMARY")
print(f"Simulation period: {CONFIG['SIMULATION_YEARS']} years")
print(f"Expected annual return: {simulation_stats['annual_return_pct']:.2f}%")
print(f"Probability of gain: {simulation_stats['probability_gain']:.1%}")
print(f"95% confidence interval: ${simulation_stats['percentile_5']:,.0f} - ${simulation_stats['percentile_95']:,.0f}")

print(f"\n📈 TOP PERFORMING SUBURBS (by median price)")
top_suburbs = geo_data.groupby('suburb')['price'].median().sort_values(ascending=False)
for suburb, price in top_suburbs.items():
    print(f"{suburb}: ${price:,.0f}")

In [ ]:
# Export results
print("💾 Exporting analysis results...")

# Save processed data
geo_data.to_csv('../data/processed/property_data_processed.csv', index=False)
valuation_analysis.to_csv('../data/outputs/valuation_analysis.csv', index=False)
simulation_results.to_csv('../data/outputs/monte_carlo_simulation.csv', index=False)

# Save model
model.save_model('../models/property_valuation_model.joblib')

# Create summary statistics file
summary_stats = {
    'analysis_date': datetime.now().isoformat(),
    'data_source': 'Free - ABS statistics + realistic generation',
    'api_required': False,
    'total_properties': len(geo_data),
    'model_performance': training_metrics,
    'simulation_stats': simulation_stats,
    'suburb_medians': geo_data.groupby('suburb')['price'].median().to_dict()
}

import json
with open('../data/outputs/analysis_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2, default=str)

print("✅ Results exported successfully!")
print("\n📁 OUTPUT FILES:")
print("- ../data/processed/property_data_processed.csv")
print("- ../data/outputs/valuation_analysis.csv")
print("- ../data/outputs/monte_carlo_simulation.csv")
print("- ../data/outputs/analysis_summary.json")
print("- ../models/property_valuation_model.joblib")
print("\n🆓 All analysis completed using free data sources only!")

## Conclusion

This **completely free** property analytics tool provides a comprehensive framework for analyzing the Australian property market without requiring any paid APIs or services.

### ✅ What This Tool Accomplishes:

1. **🆓 Free Data Sources**: Uses ABS statistics and realistic data generation - no API keys needed
2. **📊 Advanced Analytics**: Calculates key market indicators and regional statistics
3. **🤖 Machine Learning**: Builds accurate valuation models for price prediction
4. **🔍 Risk Assessment**: Identifies over/undervalued properties
5. **🔮 Forecasting**: Uses Monte Carlo simulation for future price predictions
6. **📊 Visualization**: Creates interactive charts and maps for data exploration

### 🚀 Key Benefits:

- **No API Keys Required**: 100% free to use
- **Realistic Data**: Based on actual Australian market patterns
- **Complete Workflow**: From data generation to final analysis
- **Production Ready**: Exportable results and saved models
- **Extensible**: Easy to modify for different regions or requirements

### 📈 Next Steps:

1. **Enhanced Features**: Add more sophisticated economic variables
2. **Real-time Updates**: Implement automated data refresh mechanisms
3. **Web Dashboard**: Deploy as a web application for broader access
4. **Advanced Models**: Experiment with deep learning approaches
5. **Regional Expansion**: Extend to other Australian cities or international markets

### 🛠️ Usage Notes:

- Install required packages: `pip install -r requirements.txt`
- No configuration needed - runs out of the box
- Results are automatically saved in the `data/outputs/` directory
- Modify CONFIG section for different analysis parameters

---

**🎉 Congratulations! You now have a fully functional, free property analytics system!**